https://yandex.ru/dev/metrika/doc/api2/concept/about.html

открытый счетчик яндекс метрики 44147844  
https://metrika.yandex.ru/dashboard?group=day&period=week&id=44147844

### Оценка возможности создания запроса

Первым сделаем запрос метода evaluate (англ. «оценивать»). Этот метод оценивает возможность создания запроса логов по его примерному размеру. Дело в том, что по заданным параметрам в Яндекс.Метрике может храниться слишком большой объём данных. Он будет слишком долго подготавливаться на серверах Яндекс.Метрики. Потому существуют ограничения: для одного счётчика Яндекс.Метрики суммарный размер данных в запросе не превышает 10 ГБ.

Запрос к методу evaluate выглядит так:

counterId в URL — идентификатор счётчика.

Также передаётся несколько параметров get-запроса:
- date1 — Первый день сбора статистики;
- date2 — Последний день (не может быть текущим днём);
- fields — Список полей через запятую. Этот параметр задаёт, какие поля из базы Яндекс.Метрики надо получать;
- source — Источник логов. Может принимать одно из двух значений: visits или hits. В зависимости от выбранного значения будет происходить получение либо данных по посещениям, либо данных по хитам (просмотрам определённых страниц, выполнению целей).

Осталось определиться с тем, из какого источника получать данные и какие поля выбрать. Это всегда зависит от задачи. Запросим, например, данные о посещениях с такими полями:
- ym:s:visitID — идентификатор посещения;
- ym:s:dateTime — время посещения;
- ym:s:isNewUser — пользователь зашёл на сайт впервые или уже бывал;
- ym:s:visitDuration — длительность посещения в секундах;
- ym:s:startURL — URL страницы входа;
- ym:s:clientID — идентификатор пользователя;
- ym:s:lastTrafficSource — источник трафика посещения.

Зададим все нужные параметры в соответствующих переменных:

In [ ]:
COUNTER_ID = 44147844
DATE1 = '2019-04-01'
DATE2 = '2019-05-31'
SOURCE = 'visits'
FIELDS = ['ym:s:visitID', 'ym:s:dateTime', 'ym:s:isNewUser','ym:s:visitDuration',
                 'ym:s:startURL', 'ym:s:clientID', 'ym:s:lastTrafficSource'] 

Сформируем словарь с параметрами:

In [ ]:
PARAM = {
        'date1': DATE1,
        'date2':DATE2,
        'source':SOURCE,
        'fields':','.join(FIELDS)
} 

Обратите внимание, поля FIELDS передают одной строкой. Чтобы сцепить строки из списка FIELDS вместе, вызывают метод join().

Объявим URL для запроса. Подставим идентификатор счётчика counter_id в URL-адрес метода evaluate:

In [ ]:
URL = 'https://api-metrika.yandex.ru/management/v1/counter/{counter_id}/logrequests/evaluate?'\
        .format(counter_id=COUNTER_ID) 

выполним GET-запрос и запишем результаты выполнения запроса в переменной r:

In [ ]:
r = requests.get(URL,headers={'Authorization': 'OAuth {0}'.format(TOKEN)},params=PARA

Обратите внимание, что помимо params, метод get() принимает параметр headers. Там содержатся HTTP-заголовки — служебные данные, которые передают серверу дополнительную информацию. Например, в каком формате должен быть возвращен ответ. В HTTP-заголовке 'Authorization' указали авторизационный токен.

In [ ]:
data = json.loads(r.text) 


Можно ли выполнить запрос получения данных из Logs API Яндекс.Метрики?

In [ ]:
data['log_request_evaluation']['possible'] 

Результат:  
True 

Можно!

### Создание запроса логов

Такие большие данные нельзя получить сразу! Сперва создают запрос логов: отправляют к API Яндекс.Метрики информацию о том, какой запрос хочется получить. Тогда сервера Яндекс.Метрики начнут его готовить.

Запрос на отчёт создают методом logrequests. Вот как он выглядит:

Параметры get-запроса к методу logrequests такие же, как у метода evaluate:
- date1 — Первый день сбора статистики;
- date2 — Последний день (не может быть текущим днём);
- fields — Список полей через запятую;
- source — Источник логов.

Объявим URL запроса:

In [ ]:
URL = 'https://api-metrika.yandex.ru/management/v1/counter/{counter_id}/logrequests?'\
        .format(counter_id=COUNTER_ID) 

Отправим post-запрос:

In [ ]:

r = requests.post(URL,headers={'Authorization': 'OAuth {0}'.format(TOKEN)},params=PARAM) 

In [ ]:
data = json.loads(r.text)
print(data) 

Получили те же параметры запроса, которые указали. А ещё request_id. Это идентификатор запроса логов. Сохраним его в переменной:


In [ ]:
request_id = data['log_request']['request_id'] 

In [1]:
import requests
import sys
import pprint


ids = {

    'Count_1': 44147844,
    # 'Count_2': 42219334,
    # 'Count_3': 21781912,
}

# Адрес api метода для запроса get 
# data_format = 'csv'
report_format = 'json'
url = f"https://api-metrika.yandex.net/stat/v1/data.{report_format}"
payload = {
    'metrics': 'ym:s:avgPageViews', 
    'dimensions': 'ym:s:operatingSystem',
    'date1': '2018-09-01',
    'date2': '2018-11-30',
    # 'filters': "ym:s:deviceCategory!='mobile'",
    'ids': 44147844,
    # 'accuracy': 'full',
    'pretty': True,
    # 'oauth_token': '05dd3dd84ff948fdae2bc4fb91f13e22bb1f289ceef0037',
    # 'limit': 100,
# Важный момент! Когда вы смотрите данные на сайте «Яндекс.Метрики», во всех отчетах отображаются данные без учета статистик, которые формируются при посещении сайта поисковыми и иными техническими роботами.
# А данные, возвращаемые API «Яндекс.Метрики», по умолчанию содержат всю информацию, в том числе с учетом «роботов». Разница может быть достаточно ощутимой    
    'filters' : "ym:s:isRobot=='No'"
}


for key, value in ids.items():
    payload['ids'] = value
    r = requests.get(url, params=payload)
    # pprint.pprint(r.json())
    data = str(r.json()['max'])[1:-1].split(",")
    print('total', key, data)

    




total Count_1 ['10.33894029']


In [42]:
import requests
import xlsxwriter


workbook = xlsxwriter.Workbook('/Users/sashashi/Desktop/File.xlsx')
worksheet = workbook.add_worksheet()
bold = workbook.add_format({'bold': True})
worksheet.write(0, 0, 'Название счетчика', bold)
worksheet.write(0, 1, 'Номер счетчика', bold)
worksheet.write(0, 2, 'Pageviews', bold)
worksheet.write(0, 3, 'Users', bold)
worksheet.write(0, 4, 'Visits', bold)


ids = {

    'Count_1': 49948510,
    'Count_2': 42219334,
    'Count_3': 21781912,
}

payload = {
    'metrics': 'ym:s:pageviews, ym:s:users, ym:s:visits',
    'date1': '2018-09-01',
    'date2': '2018-11-30',
    'filters': "ym:s:deviceCategory!='mobile'",
    'ids': 49948510,
    'accuracy': 'high',
    'limit': 10000,
    'pretty': True,
    'oauth_token':'AQAEA7qiTBNJAAVUsos1JB7stE6fiJV_SG11uso'
}

for key, value in ids.items():
    payload['ids'] = value
    r = requests.get('https://api-metrika.yandex.ru/stat/v1/data', params=payload)
    worksheet.write(i, 0, key)
    worksheet.write(i, 1, str(payload['ids']))
    data = str(r.json()['max'])[1:-1].split(",")
    worksheet.write(i, 2, data[0])
    worksheet.write(i, 3, data[1])
    worksheet.write(i, 4, data[2])
    payload['ids'] = value
    print('Счетчик', key, data)


workbook.close()


Счетчик Count_1 ['36551.0', ' 10112.0', ' 17309.0']
Счетчик Count_2 ['22768699.0', ' 584861.0', ' 2656932.0']
Счетчик Count_3 ['13693.0', ' 7848.0', ' 10047.0']
Счетчик Count_4 ['36551.0', ' 10112.0', ' 17309.0']
Счетчик Count_5 ['22768699.0', ' 584861.0', ' 2656932.0']
Счетчик Count_6 ['13693.0', ' 7848.0', ' 10047.0']


tapi_yandex_metrika

In [ ]:
!pip install tapi_yandex_metrika

https://github.com/pavelmaksimov/tapi-yandex-metrika

In [ ]:
from tapi_yandex_metrika import YandexMetrikaStats
ACCESS_TOKEN = "AgAAAAAJ8KAaAAYl4OT0R-7vKU3IkJuf0A0lN5M"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
params = dict(
    ids="44147844",
    metrics="ym:s:visits,ym:s:bounces",
    dimensions="ym:s:date,ym:s:startOfMonth",
    sort="ym:s:date",
    limit=10
)
result = api.stats().get(params=params)
print(result().data)

In [ ]:
from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AgAAdsf4fffaAAYl4OT0R-7vKU3IkJuf0A0lN5M"
METRIC_IDS = "54333315"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:users,ym:s:visits,ym:s:pageviews,ym:s:bounceRate,ym:s:pageDepth,ym:s:avgVisitDurationSeconds",
    dimensions = "ym:s:date,ym:s:<attribution>TrafficSource,ym:s:<attribution>SourceEngine,ym:s:gender",
    date1 = "2daysAgo",
    date2 = "yesterday",
    sort = "ym:s:date",
    accuracy="full",
    limit = 200
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
            'traffic-source':result[i]["dimensions"][1]["name"],
            'traffic-details':result[i]["dimensions"][2]["name"],
            'users':result[i]["metrics"][0],
            'visits':result[i]["metrics"][1],
            'pageviews':result[i]["metrics"][2],
            'bounceRate':result[i]["metrics"][3],
            'pageDepth':result[i]["metrics"][4],
            'avgVisitDurationSeconds':result[i]["metrics"][5]
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
#Выгрузка данных из DataFrame в Excel
df.to_excel("Трафик.xlsx",
        sheet_name='data',
        index=False)